# SocraticFlanT5 - Improved

## Introduction

In this notebook, we propose a new method to obtain image captions via the Socratic method.
This is an improved pipeline that has for goal to generate similar or improved captions using open-source and free models.

## Imports

Only a single class is needed to import all of the Image Captioning functionality.

In [1]:
# Package loading
import sys
sys.path.append('..')
from scripts.image_captioning import ImageCaptionerImprovedExtended

## Instantiate the improved captioner class

During the image captioner class instantiation, all of the helper classes such as ClipManager, ImageManager

In [2]:
image_captioner = ImageCaptionerImprovedExtended(set_type='demo')

## Run main

In [3]:
image_captioner.main()

## Load image and compute image embedding

In [4]:
image_folder = '../data/images/example_images/'
# Show the image
for img_name in image_captioner.image_manager.demo_names:
    print(f'{img_name}:')
    image_captioner.show_demo_image(img_name)
    cond = image_captioner.generated_caption_df['image_name'] == image_folder + img_name
    generate_caption = image_captioner.generated_caption_df[cond]['generated_caption'].iloc[0]
    print(f'generate caption: "{generate_caption}"\n')
    print('====================================\n')